In [2]:
import pandas as pd
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import PyPDF2
import os
from os import walk
import os.path
from os import path
import numpy as np
#!pip install py-readability-metrics
from readability import Readability
#!python -m nltk.downloader punkt

import time
from datetime import date

In [3]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text)
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [4]:
def read_pdf (file):
    try:
        pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        if pdfFile.isEncrypted:
            try:
                pdfFile.decrypt('')
                #print ('File Decrypted (PyPDF2)')
            except:
                #
                #
                # This didn't go so well on my Windows box so I just ran this in the pdf folder's cmd:
                # for %f in (*.*) do copy %f temp.pdf /Y && "C:\Program Files (x86)\qpdf-8.0.2\bin\qpdf.exe" --password="" --decrypt temp.pdf %f
                #
                #
                #
                
                command="cp "+file+" tmp/temp.pdf; qpdf --password='' --decrypt tmp/temp.pdf "+file
                os.system(command)
                #print ('File Decrypted (qpdf)')
                #re-open the decrypted file
                pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        text = ""
        for page in pdfFile.pages:
            text = text + " " + page.extractText()
        return text
    except:
        return ""


In [5]:
def parse_form(fileloc,title=None,cat=None):
    f = PyPDF2.PdfFileReader(fileloc)
    npages = f.getNumPages()
    ff = f.getFields()
    if ff:
        fields = list(ff.keys())
    else:
        fields = []
    f_per_page = len(fields)/npages
    text = read_pdf(fileloc)
    try:
        readbility = int(Readability(text).flesch_kincaid().grade_level)
    except:
        readbility = None
    
    if title is None:
        title = reCase(re.search("(.*)\n",text).group(1).strip())
    
    stats = {
            "title":title,
            "category":cat,
            "pages":npages,
            "reading grade level": readbility,
            "avg fields per page": f_per_page,
            "fields":fields
            }    
    return stats

In [6]:
files_df_ut = pd.read_csv("../data/raw/www.utcourts.gov/form_data.csv")
files_df_mi = pd.read_csv("../data/raw/www.courts.michigan.gov/form_data.csv")

df = pd.concat([files_df_ut,files_df_mi],ignore_index=True)
df["pages"] = ""
df["fields"] = ""
df["f_per_p"] = ""
df["reading"] = ""

for index,row in df.iterrows():
    if row["pages"] == "":
        try:
            stats = parse_form("../data/raw/"+row["source"]+"/forms/"+row["id"]+".pdf",row["title"],row["group"])
            df.at[index, 'pages'] = stats["pages"]
            df.at[index, 'fields'] = stats["fields"]
            df.at[index, 'f_per_p'] = stats["avg fields per page"]
            df.at[index, 'reading'] = stats["reading grade level"]
            #print(index)
        except:
            print("error: "+"../data/raw/"+row["source"]+"/forms/"+row["id"]+".pdf")
        
df

error: ../data/raw/www.utcourts.gov/forms/97b5d7abdb714d6d833a2725ba4f4c17.pdf
error: ../data/raw/www.utcourts.gov/forms/c6bef516c1dd4f4fb31cf4d824245f44.pdf


,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,f_per_p,reading
0,ed1efd08e3304bdaa887db753509d666,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-03,2,[],0.0,10
1,28ab1fb8df5c477cb4353e44f6d5a404,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-03,2,[],0.0,11
2,279083cdac044d22b873a6a6ab8d9a14,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-03,5,[],0.0,15
3,ebad2caa6be0458d9e5c75635a7f66c2,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-03,3,[],0.0,10
4,92932e4f938440ce8b93b6434aedfc3d,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-03,1,[],0.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...
1524,94be7e4a467c46c5b8f18a81df4c3371,MI,www.courts.michigan.gov,Claim of Appeal of Right/Request to Withdraw A...,civil infraction,https://www.courts.michigan.gov/4a2ca8/siteass...,cia05.pdf,2021-11-03,1,"[disno, caseno, judge, address, telno, name, d...",40.0,20
1525,9d46a79a162844ebb78fc592873c027b,MI,www.courts.michigan.gov,"Default Judgment, Civil Infraction",civil infraction,https://www.courts.michigan.gov/4a7328/siteass...,cia07.pdf,2021-11-03,1,[],0.0,14
1526,6d97d3ca4103413b8e161013c26943d6,MI,www.courts.michigan.gov,"Judgment, Civil Infraction",civil infraction,https://www.courts.michigan.gov/4a7336/siteass...,cia02.pdf,2021-11-03,2,"[STATE OF MICHIGAN JUDICIAL DISTRICT COUNTY, S...",20.5,9
1527,480c76eb519d4242a7f44722bb7fbea6,MI,www.courts.michigan.gov,Motion to Set Aside Default Judgment and Order...,civil infraction,https://www.courts.michigan.gov/4a2cb4/siteass...,cia04.pdf,2021-11-03,1,"[disno, caseno, judge, address, telno, dname, ...",22.0,17


In [94]:
df.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    

In [95]:
os.system('cp ../data/processed/form_data.csv docs/forms/form_data.csv')
os.system('cp ../data/raw/www.utcourts.gov/forms/* docs/forms/')
os.system('cp ../data/raw/www.courts.michigan.gov/forms/* docs/forms/')

0